# Pandas Review: UN Population Data

**Attribution**: _This notebook is based on a notebook created by Dave Backus, Chase Coleman, Brian LeBlanc, and Spencer Lyon for the NYU Stern Data Bootcamp course_

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from urllib.parse import urljoin

%matplotlib inline

## UN Population Data

We will look at the UN's [population data](http://esa.un.org/unpd/wpp/Download/Standard/Population/). In this notebook, we will focus specifically the age distribution of the population.


### Projection Variants

The population numbers for a particular year are reported as corresopnding to 1 July 2020. There will be two types of population numbers that we discuss:

* *estimates*: Estimates of the population at a point in the past
* *projections*: Forecasts of what the population might be for a given year

Furthermore, the UN will provide various versions of the projections using different modeling assumptions. We will consider three of their population models:

1. *Low variant*: Assumes a low fertility with normal mortality and normal migration.
2. *Medium variant*: Assumes a medium fertility with normal mortality and normal migration.
3. *High variant*: Assumes a high fertility with normal mortality and normal migration

### Loading the data 

We start, as usual, by loading the data.  This will take a minute because the file is about 10 MB in size.

In [ ]:
un_base_url = "https://population.un.org/"
path_to_file = "wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/"
filename = ("WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.xlsx")

url = urljoin(un_base_url, path_to_file+filename)

In [ ]:
cols_to_read = [
    "Variant", "Region, subregion, country or area *", "Country code",
    "Type", "Reference date (as of 1 July)"
]

age_cols = [f"{5*i}-{5*i+4}" for i in range(20)]
age_cols.append("100+")

cols_to_read.extend(age_cols)

sheets = pd.read_excel(
    url, sheet_name=["ESTIMATES", "LOW VARIANT", "MEDIUM VARIANT", "HIGH VARIANT"],
    skiprows=16, usecols=cols_to_read, na_values=["..."]
)

est = sheets["ESTIMATES"]
low_proj = sheets["LOW VARIANT"]
med_proj = sheets["MEDIUM VARIANT"]
hgh_proj = sheets["HIGH VARIANT"]

# Combine all of the datasets into one
df = pd.concat([est, low_proj, med_proj, hgh_proj], axis=0, ignore_index=True)

What does our data contain?


In [ ]:
df.head()

In [ ]:
print("Dimensions and dtypes of estimates:")
print(df.shape)
print(df.dtypes)

### Clean data

The first step to using this data is to make sure that the data is clean.

In [ ]:
df.head()

**Geography types**

There are currently many different geography types including `World`, `Label/Separator`, `Development Group`, `Income Group`, ..., and `Country/Area`.

For what we're going to do today, we're going to focus on country level information.

In [ ]:
df = df.query("Type == 'Country/Area'")

df.head()


**Renaming**

Many column names aren't easily typable or usable. We will resolve this by renaming the columns and will use the conventions below for renaming our columns:

* Short and memorable (using standard conventions where possible)
* Lower case with underscores to separate words when needed
* Consistent <- Most important

In [ ]:
renamers = {
    "Variant": "variant",
    "Region, subregion, country or area *": "country",
    "Country code": "alpha3",
    "Type": "type",
    "Reference date (as of 1 July)": "year"
}

# Rename the columns and drop type column (since we've already restricted
# to just countries)
df = df.rename(
    columns=renamers
).drop(
    ["type"], axis=1
)

df.head()


**Reshaping**

The data currently has one column for each age group. We'll update the data to be a long-form rather than wide-form to make it "tidy"

In [ ]:
df = df.melt(id_vars=["variant", "country", "alpha3", "year"], var_name="age")

df.head()


**Population sizes**

Let's focus on countries that had more than 50,000,000 people in 2010, 2015, and 2020 according to the estimates data.

In order to do this, we're going to use a pivot table to determine how many people were in each country during each year and then use boolean selection

In [ ]:
pop_sizes = (
    df.query("variant == 'Estimates'")
      .pivot_table(index="country", columns="year", values="value", aggfunc="sum")
)

# Get list of the countries with more than 50m people
countries_pop_gt_50m_bool = (pop_sizes.loc[:, 2010:2020] > 50_000).all(axis=1)
countries_pop_gt_50m = countries_pop_gt_50m_bool.index[countries_pop_gt_50m_bool].tolist()

In [ ]:
df_gt_50m = df.query("country in @countries_pop_gt_50m")

df_gt_50m.head()


### Visualizations

Let's now begin visually exploring our data.

**Plotting age distribution**

Below, we'll begin plotting the age distributions of certain countries in different years. Let's start by plotting the age distribution in China for 1960, 1990, 2020, 2050, and 2080.

To do this, we  will need `year` on the index and `age` as the columns.

In [ ]:
china_sel = "country == 'China' & (variant=='Estimates' | variant=='Medium variant')"
df_china = (
    df.query(china_sel)
      .pivot_table(index="age", columns="year", values="value")
      .loc[:, range(1960, 2100, 30)]
)


In [ ]:
ax = df_china.plot(
    kind="bar", figsize=(10, 8), subplots=True, sharey=True,
    legend=False
)

fig = ax[0].get_figure()
fig.tight_layout()

What's wrong with the graph?

In [ ]:
ax = df_china.loc[age_cols, :].plot(
    kind="bar", figsize=(10, 8), subplots=True, sharey=True,
    legend=False
)

fig = ax[0].get_figure()
fig.tight_layout()

Maybe we want to see the fraction of the population in each bin rather than the number of people

In [ ]:
ax = (
    df_china.divide(df_china.sum(axis=0), axis=1)
            .loc[age_cols, :]
            .plot(
                kind="bar", figsize=(10, 8), subplots=True,
                sharey=True, legend=False, ylim=(0.0, 0.2),
                title="Population Share by Age Bracket"
            )
)

for (i, _ax) in enumerate(ax):
    _population_m = df_china.iloc[:, i].sum()/1000
    _title = f"{df_china.columns[i]}: {_population_m:.2f} million"
    _ax.set_title("")
    _ax.set_title(_title, loc="right")
    _ax.spines["right"].set_visible(False)
    _ax.spines["top"].set_visible(False)

fig = ax[0].get_figure()
fig.tight_layout()


If we wanted a similar graph for another country, we could potentially copy and paste what we wrote, but, if we made minor changes then we could potentially introduce unexpected errors when trying to write them.

Instead, let's write a function that creates a graph like this.

In [ ]:
def age_distribution_graph(data, country, years, forecast_variant="Medium variant"):
    # Subset the data
    sel = f"country == '{country}' & (variant=='Estimates' | variant=='{forecast_variant}')"
    plot_df = (
        data.query(sel)
            .pivot_table(index="age", columns="year", values="value")
            .loc[:, years]
    )
    
    # Create the plot
    ax = (
        plot_df.divide(plot_df.sum(axis=0), axis=1)
            .loc[age_cols, :]
            .plot(
                kind="bar", figsize=(10, 8), subplots=True,
                sharey=True, legend=False, ylim=(0.0, 0.2),
                title=None
            )
    )

    for (i, _ax) in enumerate(ax):
        _population_m = plot_df.iloc[:, i].sum()/1000
        _title = f"{plot_df.columns[i]}: {_population_m:.2f} million"
        _ax.set_title("")
        _ax.set_title(_title, loc="right")
        _ax.spines["right"].set_visible(False)
        _ax.spines["top"].set_visible(False)

    fig = ax[0].get_figure()
    ax[0].set_title(f"Population Share by Age Bracket for {country}", loc="left")
    fig.tight_layout()
    
    return fig


### Population graphs

In [ ]:
years_of_interest = [1980, 1990, 2000, 2020, 2050, 2100]

**China**

In [ ]:
age_distribution_graph(df, "China", years_of_interest, "Low variant");

In [ ]:
age_distribution_graph(df, "China", years_of_interest, "Medium variant");

In [ ]:
age_distribution_graph(df, "China", years_of_interest, "High variant");

**India**

In [ ]:
age_distribution_graph(df, "India", years_of_interest, "Low variant");

In [ ]:
age_distribution_graph(df, "India", years_of_interest, "Medium variant");

In [ ]:
age_distribution_graph(df, "India", years_of_interest, "High variant");

**United States of America**

In [ ]:
age_distribution_graph(df, "United States of America", years_of_interest, "Low variant");

In [ ]:
age_distribution_graph(df, "United States of America", years_of_interest, "Medium variant");

In [ ]:
age_distribution_graph(df, "United States of America", years_of_interest, "High variant");

## Why do we care about age distribution?

Lots of reasons, but one that we (as young working-age adults) should be particularly interested in is social security programs.


_Definition: **Dependency ratio**_

> The (old-age) dependency ratio relates the number of persons aged 65 or over per 100 persons aged 15-64.

We care about this measurement because it tells us roughly how many working age people there are to support those who have stopped working. As the population distribution shifts to the right (and if nothing changes), there will be fewer individuals to support the social security programs that support the old (non-working) population.

In [ ]:
# Compute old-age dependency ratio for countries with more than
# 50,000,000 people
working_age = [f"{5*i}-{5*i + 4}" for i in range(20) if 5*i >= 15 and 5*i<65]
old_age = [f"{5*i}-{5*i + 4}" for i in range(20) if 5*i>=65] + ["100+"]

df_gt_50m.loc[:, "age_classification"] = "young"
wa = df_gt_50m["age"].isin(working_age)
df_gt_50m.loc[wa, "age_classification"] = "work"
oa = df_gt_50m["age"].isin(old_age)
df_gt_50m.loc[oa, "age_classification"] = "old"

dr = (
    df_gt_50m.query("variant == 'Estimates' | variant == 'Medium variant'")
             .pivot_table(
                 index=["country", "year"],
                 columns="age_classification", values="value"
             )
)

dr.loc[:, "dep_ratio"] = 100 * (dr.loc[:, "old"] / dr.loc[:, "work"])

dr = dr["dep_ratio"].unstack(level="country")

dr.head()


**Sustainable dependency ratio**

Obviously the age of retirement could be adjusted which would make the dependency ratio, as we've calculated it, a worse measure of dependence. However, just as a thought experiment, let's suppose that the dependency ratio that is sustainable long term (i.e. the payments to social security are roughly the same as the social security tax collected) is 50 (which means that there are 2 workers for each person over 65).

Of our countries with over 50 million citizens, how many countries will be sustainable in 2080? In 2100?

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

dr.plot(kind="line", ax=ax, legend=False)
ax.hlines(50, 1950, 2100, color="k", linewidth=2.0, linestyle="--")

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.set_title("Dependency Ratios for countries with 50m+ citizens")

In [ ]:
dr_2080 = dr.loc[2080, :]
dr_2080.index[dr_2080 < 50]

In [ ]:
dr_2100 = dr.loc[2100, :]
dr_2100.index[dr_2100 < 50]